In [13]:
import pandas as pd
pd.options.mode.chained_assignment = None
import altair as alt
alt.renderers.enable('mimetype')
alt.data_transformers.disable_max_rows()
import networkx as nx
import os

In [14]:
user_df = pd.read_csv("../data/entity_files/users_dataset.csv")
repo_df = pd.read_csv("../data/large_files/entity_files/repos_dataset.csv", low_memory=False)
org_df = pd.read_csv("../data/entity_files/orgs_dataset.csv", low_memory=False)
core_users = pd.read_csv("../data/derived_files/initial_core_users.csv")
core_repos = pd.read_csv("../data/derived_files/initial_core_repos.csv")
core_orgs = pd.read_csv("../data/derived_files/initial_core_orgs.csv")

- user-user interactions:
  - org_members
  - user_orgs
  - user_followers
  - user_following

- user-repo interactions:
  - user_repos
  - contirbutors_repos
  - starrers_repos
  - subscribers_repos
  - forkers_repos
  - committers_repos
  - 

In [96]:
dfs = []
for dir, subdir, files in os.walk("../data/join_files/"):
    for f in files:
        if ('search' not in f) and (f.endswith(".csv")):
            entity_type = f.split("_")[0]
            print(f)
            df = pd.read_csv(os.path.join(dir, f), nrows=10)
            cols = df.columns
            cols = [col for col in cols if any(x in col for x in ['full_name', 'login'])]
            cols = [col for col in cols if ('.' not in col) or ('user.login' in col)]
            if len(cols) > 2:
                cols = ['user.login', 'repo_full_name']
            if len(cols) == 1:
                cols = cols + ['author.login']
            df_dict = {}
            df_dict['entity_type'] = entity_type
            df_dict['source'] = cols[0]
            df_dict['target'] = cols[1]
            df_dict['file_name'] = os.path.join(dir, f)
            dfs.append(df_dict)

for dir, subdir, files in os.walk("../data/large_files/join_files/"):
    for f in files:
        if ('search' not in f) and (f.endswith(".csv")):
            entity_type = f.split("_")[0]
            print(f)
            df = pd.read_csv(os.path.join(dir, f), nrows=10)
            cols = df.columns
            cols = [col for col in cols if any(x in col for x in ['full_name', 'login'])]
            cols = [col for col in cols if ('.' not in col) or ('user.login' in col)]
            if len(cols) > 2:
                cols = ['user.login', 'repo_full_name']
            if len(cols) == 1:
                cols = cols + ['author.login']
            df_dict = {}
            df_dict['entity_type'] = entity_type
            df_dict['source'] = cols[0]
            df_dict['target'] = cols[1]
            df_dict['file_name'] = os.path.join(dir, f)
            dfs.append(df_dict)

org_repos_join_dataset.csv
org_members_join_dataset.csv
repo_subscribers_join_dataset.csv
repo_stargazers_join_dataset.csv
repo_contributors_join_dataset.csv
user_orgs_join_dataset.csv
user_repos_join_dataset.csv
user_following_join_dataset.csv
user_subscriptions_join_dataset.csv
issues_comments_join_dataset.csv
repo_commits_join_dataset.csv
user_followers_join_dataset.csv
user_starred_join_dataset.csv
user_organizations_join_dataset.csv
repo_issues_join_dataset.csv
repo_pulls_join_dataset.csv
pulls_comments_join_dataset.csv
repo_forks_join_dataset.csv


In [97]:
cols_df = pd.DataFrame(dfs)

In [98]:
cols_df

,entity_type,source,target,file_name
0,org,full_name,org_login,../data/join_files/org_repos_join_dataset.csv
1,org,login,org_login,../data/join_files/org_members_join_dataset.csv
2,repo,login,repo_full_name,../data/join_files/repo_subscribers_join_datas...
3,repo,user.login,repo_full_name,../data/join_files/repo_stargazers_join_datase...
4,repo,login,repo_full_name,../data/join_files/repo_contributors_join_data...
5,user,login,user_login,../data/join_files/user_orgs_join_dataset.csv
6,user,full_name,user_login,../data/large_files/join_files/user_repos_join...
7,user,login,user_login,../data/large_files/join_files/user_following_...
8,user,full_name,user_login,../data/large_files/join_files/user_subscripti...
9,issues,user.login,repo_full_name,../data/large_files/join_files/issues_comments...


: 

In [93]:
test = pd.read_csv("../data/large_files/join_files/repo_commits_join_dataset.csv", nrows=10)

In [95]:
test[0:1].to_dict()

{'sha': {0: 'a4cb55355ba8fbe6cf19691600d19524fc110aef'},
 'node_id': {0: 'MDY6Q29tbWl0MjgzOTAzMjQxOmE0Y2I1NTM1NWJhOGZiZTZjZjE5NjkxNjAwZDE5NTI0ZmMxMTBhZWY='},
 'url': {0: 'https://api.github.com/repos/wildmary/Portfolio-DH/commits/a4cb55355ba8fbe6cf19691600d19524fc110aef'},
 'html_url': {0: 'https://github.com/wildmary/Portfolio-DH/commit/a4cb55355ba8fbe6cf19691600d19524fc110aef'},
 'comments_url': {0: 'https://api.github.com/repos/wildmary/Portfolio-DH/commits/a4cb55355ba8fbe6cf19691600d19524fc110aef/comments'},
 'parents': {0: "[{'sha': '3bb3421fa786b5762d2cef6af5846ce0254a8bd1', 'url': 'https://api.github.com/repos/wildmary/Portfolio-DH/commits/3bb3421fa786b5762d2cef6af5846ce0254a8bd1', 'html_url': 'https://github.com/wildmary/Portfolio-DH/commit/3bb3421fa786b5762d2cef6af5846ce0254a8bd1'}]"},
 'commit.author.name': {0: 'Mary Levchenko'},
 'commit.author.email': {0: 'wildmary@yandex.ru'},
 'commit.author.date': {0: '2020-07-31T02:24:21Z'},
 'commit.committer.name': {0: 'GitHub'},
 'co

In [62]:
merged_df.drop(columns=['updated_org_login'], inplace=True)

In [66]:
merged_df = merged_df.drop_duplicates(subset=['login', 'org_login'])

In [67]:
merged_df.to_csv("../data/join_files/org_members_join_dataset.csv", index=False)

In [38]:
contributors_df = pd.read_csv("../data/join_files/repo_contributors_join_dataset.csv")
filtered_contributors_df = contributors_df[((contributors_df.login.isin(core_users.login)) | (contributors_df.login.isin(core_orgs.login))) & (contributors_df.repo_full_name.isin(core_repos.full_name))]

In [39]:
filtered_contributors_df

,login,id,node_id,avatar_url,gravatar_id,url,html_url,followers_url,following_url,gists_url,...,received_events_url,type,site_admin,contributions,repo_id,repo_url,repo_html_url,repo_full_name,contributors_url,repo_query_time
1,jhroy,1175329,MDQ6VXNlcjExNzUzMjk=,https://avatars.githubusercontent.com/u/117532...,NaN,https://api.github.com/users/jhroy,https://github.com/jhroy,https://api.github.com/users/jhroy/followers,https://api.github.com/users/jhroy/following{/...,https://api.github.com/users/jhroy/gists{/gist...,...,https://api.github.com/users/jhroy/received_ev...,User,False,37,85891959.0,https://api.github.com/repos/jhroy/tuto-pandas,https://github.com/jhroy/tuto-pandas,jhroy/tuto-pandas,https://api.github.com/repos/jhroy/tuto-pandas...,2022-12-16
18,malamatenia,92155767,U_kgDOBX4vdw,https://avatars.githubusercontent.com/u/921557...,NaN,https://api.github.com/users/malamatenia,https://github.com/malamatenia,https://api.github.com/users/malamatenia/follo...,https://api.github.com/users/malamatenia/follo...,https://api.github.com/users/malamatenia/gists...,...,https://api.github.com/users/malamatenia/recei...,User,False,88,423510743.0,https://api.github.com/repos/PSL-Chartes-HTR-S...,https://github.com/PSL-Chartes-HTR-Students/HN...,PSL-Chartes-HTR-Students/HN2021-Boccace,https://api.github.com/repos/PSL-Chartes-HTR-S...,2022-12-16
26,Ammeister,72987654,MDQ6VXNlcjcyOTg3NjU0,https://avatars.githubusercontent.com/u/729876...,NaN,https://api.github.com/users/Ammeister,https://github.com/Ammeister,https://api.github.com/users/Ammeister/followers,https://api.github.com/users/Ammeister/followi...,https://api.github.com/users/Ammeister/gists{/...,...,https://api.github.com/users/Ammeister/receive...,User,False,27,532646221.0,https://api.github.com/repos/Ammeister/AP_ENC,https://github.com/Ammeister/AP_ENC,Ammeister/AP_ENC,https://api.github.com/repos/Ammeister/AP_ENC/...,2022-12-16
61,anisa-hawes,87070441,MDQ6VXNlcjg3MDcwNDQx,https://avatars.githubusercontent.com/u/870704...,NaN,https://api.github.com/users/anisa-hawes,https://github.com/anisa-hawes,https://api.github.com/users/anisa-hawes/follo...,https://api.github.com/users/anisa-hawes/follo...,https://api.github.com/users/anisa-hawes/gists...,...,https://api.github.com/users/anisa-hawes/recei...,User,False,817,12707869.0,https://api.github.com/repos/programminghistor...,https://github.com/programminghistorian/jekyll,programminghistorian/jekyll,https://api.github.com/repos/programminghistor...,2022-12-16
68,fredgibbs,515448,MDQ6VXNlcjUxNTQ0OA==,https://avatars.githubusercontent.com/u/515448...,NaN,https://api.github.com/users/fredgibbs,https://github.com/fredgibbs,https://api.github.com/users/fredgibbs/followers,https://api.github.com/users/fredgibbs/followi...,https://api.github.com/users/fredgibbs/gists{/...,...,https://api.github.com/users/fredgibbs/receive...,User,False,240,12707869.0,https://api.github.com/repos/programminghistor...,https://github.com/programminghistorian/jekyll,programminghistorian/jekyll,https://api.github.com/repos/programminghistor...,2022-12-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4720,arockenberger,12702667,MDQ6VXNlcjEyNzAyNjY3,https://avatars.githubusercontent.com/u/127026...,NaN,https://api.github.com/users/arockenberger,https://github.com/arockenberger,https://api.github.com/users/arockenberger/fol...,https://api.github.com/users/arockenberger/fol...,https://api.github.com/users/arockenberger/gis...,...,https://api.github.com/users/arockenberger/rec...,User,False,46,419631237.0,https://api.github.com/repos/humsamdigiforsk/o...,https://github.com/humsamdigiforsk/organisatorisk,humsamdigiforsk/organisatorisk,https://api.github.com/repos/humsamdigiforsk/o...,2022-12-16
4729,jessesadler,2695312,MDQ6VXNlcjI2OTUzMTI=,https://avatars.githubusercontent.com/u/269531...,NaN,https://api.github.com/users/jessesadler,https://github.com/jessesadler,https://api.github.com/users/jessesadler/follo...,https://api.g

In [40]:
edges = filtered_contributors_df[['login', 'repo_full_name']]
edges = list(zip(edges['login'], edges['repo_full_name']))

In [41]:
bipartite_graph = nx.Graph()

In [42]:
user_nodes = core_users['login'].tolist()
repo_nodes = core_repos['full_name'].tolist()
org_nodes = core_orgs['login'].tolist()
user_nodes = user_nodes + org_nodes

In [32]:
len(user_nodes), len(repo_nodes), len(edges)

(793, 2264, 4983)

In [43]:
bipartite_graph.add_nodes_from(user_nodes, bipartite=0)
bipartite_graph.add_nodes_from(repo_nodes, bipartite=1)
bipartite_graph.add_edges_from(edges)

In [44]:
print('connected?', nx.is_connected(bipartite_graph))
print('bipartite?', nx.is_bipartite(bipartite_graph))

connected? False
bipartite? True


In [45]:
nx.write_gexf(bipartite_graph,"../data/derived_files/bipartite_graph.gexf")

In [2]:
test = pd.read_excel("../data/private_data/Appendix A publication_raw_chun_llc_dhq.xlsx")

In [9]:
test2 = pd.read_excel("../data/private_data/Appendix E co-retweet network.xlsx")

In [10]:
test2

,Unnamed: 0,label,handle,no. tweet,no. retweet received,no. nonself retweet received,retweeted %,betweenness centrality,gender,country,most retweeting hashtag,most retweeted hashtag,account create year
0,1,ADHO,ADHOrg,2595,3028,2590,0.998073,179742.715164,U,Unknown,digitalhumanities(114); dh2014(87); dh2015(83)...,digitalhumanities(665); dh2016(406); dh2017(27...,2013
1,966,NEH,NEHgov,3232,4093,4145,1.282488,103.811686,U,United States,humanities(77); nehgrant(54); cubavisit(32); n...,nehgrant(504); neh(271); humanities(196); otd(...,2010
2,340,melissa terras,melissaterras,3202,2219,3899,1.217676,589.969181,F,United Kingdom,digitalhumanities(16); dh2017(15); bldigital(6...,digitalhumanities(548); dh2017(158); dh2014(12...,2009
3,4,aaDH,aaDHumanities,1199,250,158,0.131776,121131.629443,U,Australia,digitalhumanities(92); dha2016(66); humanities...,dha2016(83); digitalhumanities(79); eresearch(...,2015
4,223,Bethany Nowviskie,nowviskie,3222,2463,4366,1.355059,1266.470119,F,United States,dlfforum(233); endangereddata(64); dlflac(54);...,dlfforum(357); dh2013(113); endangereddata(109...,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3146,3147,yhkondo,yhkondo,830,0,8,0.009639,0.000000,M,Japan,androidapp(1); asteriskdb0090(1); epub_tw(1); ...,0,2009
3147,3148,Yuji Shinozaki,ys2n,284,0,0,0.000000,0.000000,M,Unknown,0,0,2008
3148,3149,Yutzu,yutzudotcom,136,6,8,0.058824,0.000000,F,Canada,0,brazil(1); curator(1); historian(1); librarian...,2010
3149,3150,Zeus Oratrios,Zeus_Kairos,1923,0,0,0.000000,0.000000,M,Unknown,notav(11); venezia(9); art(8); fb(6); terremot...,0,2009


In [8]:
test

,Authors,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,Page count,...,Conference code,ISSN,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Source,EID
0,"Spencer M., Wachtel K., Howe C.J.",Representing multiple pathways of textual flow...,2004,Computers and the Humanities,38,1,NaN,1,14,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,-,Article,Scopus,2-s2.0-26844469206
1,"Collins J., Kaufer D., Vlachos P., Butler B., ...",Detecting collaborations in text,2004,Computers and the Humanities,38,1,NaN,15,36,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,-,Article,Scopus,2-s2.0-28344440777
2,"Can F., Patton J.M.",Change of writing style with time,2004,Computers and the Humanities,38,1,NaN,61,82,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,-,Article,Scopus,2-s2.0-33845685623
3,"Pohl M., Purgathofer P.",Hypertext writing profiles and visualisation,2004,Computers and the Humanities,38,1,NaN,83,105,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,-,Article,Scopus,2-s2.0-33845692183
4,Wilks Y.,On the ownership of text,2004,Computers and the Humanities,38,2,NaN,115,127,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,-,Article,Scopus,2-s2.0-33845693781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,[No author name available],The Thomas Mann project: A progress report,1966,Computers and the Humanities,1,3,NaN,65,71,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,Comput Hum,Article,Scopus,2-s2.0-34250504503
1031,[No author name available],The computer and Plato's Seventh letter,1966,Computers and the Humanities,1,3,NaN,72,73,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,Comput Hum,Article,Scopus,2-s2.0-34250519129
1032,[No author name available],The new center for research at the University ...,1966,Computers and the Humanities,1,3,NaN,73,74,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,Comput Hum,Article,Scopus,2-s2.0-34250514213
1033,[No author name available],Literary works in machine-readable form,1966,Computers and the Humanities,1,3,NaN,75,102,NaN,...,NaN,104817.0,NaN,NaN,NaN,English,Comput Hum,Article,Scopus,2-s2.0-34250541874
